In [ ]:
%%writefile /kaggle/working/README.txt

     _ _       _     _   _   
    | (_)     | |   | | | |  
  __| |_ _ __ | |__ | |_| |_ 
 / _` | | '_ \| '_ \| __| __|
| (_| | | | | | | | | |_| |_ 
 \__,_|_|_| |_|_| |_|\__|\__| @ randrise.com
--------------------------------------------
 Bidirectional Brute Force w/ Move Priority
--------------------------------------------


============================================
              REFERENCES
  ---------------------------------------

# Bidirectional Brute Force
# https://www.kaggle.com/code/davidlist/bidirectional-brute-force

The difficulty with brute force methods in most all of these problems is the extent with 
which the number of permutations explodes exponentially with the distance from any given state. 
However, by proceeding outwards from both the initial state and the solution state with 
the hope of meeting in the middle, it's possible to limit the permutations enough 
to find optimal solutions for a few of the puzzles.
   

============================================
              PROBLEM TO FACE
  ---------------------------------------

When using brute force, we face the problem that too many states are stored. Even we save data to disk,
we face 'No space left' problem.
   

============================================
              HOW TO SOLVE
  ---------------------------------------

Before storing state, we check if state is nearly related to solution state or not, new move having 
score larger than sample solution or not. We only store best states, so that needs of RAM is decreased.
   
   

In [ ]:
import pandas as pd
import numpy as np
import ast
import os
import time

In [ ]:
PUZZLE_FILE = '/kaggle/input/santa-2023/puzzles.csv'
PUZZLE_INFO_FILE = '/kaggle/input/santa-2023/puzzle_info.csv'
SAMPLE_FILE = '/kaggle/input/santa-2023/sample_submission.csv'
SAMPLE_FILE = '/kaggle/input/santa-2023-solutions-ensemble/submission.csv'

RUN_TIME = 60 * 60 * 11.5
START_TIME = time.time()
TIMEOUT = 60 * 60 * 11.5
TRACK_TIME = 30
MAX_CNT_RATIO = 200
MAX_CNT_RATIO = None
DEBUG_TRACK = True
DEBUG_LOOP = True

INCLUDES = None
INCLUDES = [368]
EXCLUDES = None
MIN_SIZE = None
MAX_SIZE = None
MAX_SIZE = None


In [ ]:
data_df = pd.read_csv(PUZZLE_FILE)
info_df = pd.read_csv(PUZZLE_INFO_FILE)
sample_df = pd.read_csv(SAMPLE_FILE)        

In [ ]:
def init_reverse_moves(moves):
    new_moves = {}
    
    for m in moves.keys():
        new_moves[m] = moves[m]
        xform = moves[m]
        m_inv = '-' + m
        xform_inv = len(xform) * [0]
        for i in range(len(xform)):
            xform_inv[xform[i]] = i
        new_moves[m_inv] = xform_inv

    return new_moves

def apply_move(move, state):

    m = move
    s = state.split(';')

    move_list = moves[m]
    new_state = []
    for i in move_list:
        new_state.append(s[i])
    s = new_state

    return ';'.join(s)

def reverse_move(move, state):
    m = move[1:] if move[0] == '-' else '-' + move
    return apply_move(m, state)

def expand(paths, reverse=False):
    global moves, source_paths, dest_paths, initial_state, solution_state, initial_score, solution_score, prv_size    
    start_tm = time.time() - TRACK_TIME
    states = list(paths.keys())
    initial_min = initial_score
    solution_min = solution_score
    
    fnd_cnt = 0
    loop_cnt = 0
    loop_max = len(initial_state.split(';')) * 10
    loop_gap = 0
    while fnd_cnt == 0 and ((reverse == False and initial_min >= initial_score) or (reverse == True and solution_min >= solution_score)) and loop_cnt < loop_max:
        cnt = 0
        for s in states:
            cnt += 1
            if time.time() - start_tm > TRACK_TIME:
                if DEBUG_TRACK:
                    print('=> [A] ' + str(cnt) + ' / ' + str(len(states)) + ' --> ' + str(initial_score) + ', ' + str(solution_score) + ' -> ' + str(initial_min) + ', ' + str(solution_min) + ' ---> ' + str(len(source_paths.keys())) + ' / ' + str(len(dest_paths.keys())))
                start_tm = time.time()

            if len(paths[s]) + 1 > prv_size:
                continue
                
            for m in moves:
                if reverse:
                    next_s = reverse_move(m, s)
                    score = get_solution_score(next_s)
                    if score < solution_score + loop_gap:
                        if not next_s in paths:
                            paths[next_s] = paths[s] + [m]
                            if score < solution_min:
                                solution_min = score
                            fnd_cnt += 1
                else:
                    next_s = apply_move(m, s)
                    score = get_initial_score(next_s)
                    if score < initial_score + loop_gap:
                        if not next_s in paths:
                            paths[next_s] = paths[s] + [m]
                            if score < initial_min:
                                initial_min = score
                            fnd_cnt += 1

            overlap = list(set(source_paths.keys()).intersection(set(dest_paths.keys())))
            if len(overlap) > 0:
                initial_score = initial_min
                solution_score = solution_min
                return

        loop_gap += 1
        if loop_gap > loop_max:
            loop_gap = loop_max
        loop_cnt += 1

    initial_score = initial_min
    solution_score = solution_min
            
def get_initial_score(state):
    global solution_state    
    return sum(not(s == t) for s, t in zip(solution_state.split(';'), state.split(';')))

def get_solution_score(state):
    global initial_state    
    return sum(not(s == t) for s, t in zip(initial_state.split(';'), state.split(';')))


In [ ]:
def solve(pid, prev_solution):
    global moves, source_paths, dest_paths, initial_state, solution_state, initial_score, solution_score, prv_size
    
    ddf = data_df[data_df['id'] == pid]
    puzzle_type = ddf['puzzle_type'].iloc[0]
    solution_state = ddf['solution_state'].iloc[0]
    initial_state = ddf['initial_state'].iloc[0]
    num_wildcards = ddf['num_wildcards'].iloc[0]

    idf = info_df[info_df['puzzle_type'] == puzzle_type]
    allowed_moves = idf['allowed_moves'].iloc[0]    
    moves = ast.literal_eval(allowed_moves)

    moves = init_reverse_moves(moves)

    initial_score = len(solution_state.split(';'))
    solution_score = len(solution_state.split(';'))
        
    prv_size = len(prv_solution.split('.'))
    
    max_cnt = None
    if MAX_CNT_RATIO is not None:
        max_cnt = solution_score * MAX_CNT_RATIO
    
    source_paths = {
        initial_state: []
    }
    dest_paths = {
        solution_state: []
    }

    start_time = time.time()
    solution = None
    count = 0
    while time.time() - START_TIME < RUN_TIME and time.time() - start_time < TIMEOUT:
        count += 1

        if max_cnt is not None and count > max_cnt:
            break
            
        if count % 2:
            expand(source_paths)
        else:
            expand(dest_paths, reverse=True)

        overlap = list(set(source_paths.keys()).intersection(set(dest_paths.keys())))
        if DEBUG_LOOP:
            print('=> [B] ' + str(count) + ' --> ' + str(initial_score) + ' : ' + str(solution_score) + ' --> ' + str(len(source_paths.keys())) + ' / ' + str(len(dest_paths.keys())))
        if len(overlap) > 0:
            mn_score = 10000000
            mn_sol = None
            for oi in range(len(overlap)):
                oe = overlap[oi]
                sol = '.'.join(source_paths[oe] + list(reversed(dest_paths[oe])))
                sz = len(sol.split('.'))
                if sz < mn_score:
                    mn_score = sz
                    mn_sol = sol
            solution = mn_sol
            break
            
    if solution is not None:
        bsz = len(prv_solution.split('.'))
        asz = len(solution.split('.'))
        if asz == bsz:
            print('[' + str(pid) + '] Same : ' + str(bsz) + ' -> ' + str(asz))
        elif asz < bsz:
            print('[' + str(pid) + '] Decrease : ' + str(bsz) + ' -> ' + str(asz))            
        else:
            print('[' + str(pid) + '] Increase : ' + str(bsz) + ' -> ' + str(asz))            
            
    return solution

In [ ]:
moves = None
source_paths = None
dest_paths = None
initial_state = None
solution_state = None
initial_score = None
solution_score = None
prv_size = None

In [ ]:
grows = []
all_ids = sample_df['id'].unique()
for pid in all_ids:
    if INCLUDES is not None and pid not in INCLUDES:
        print('=> [' + str(pid) + '] Not included!')
        continue
    if EXCLUDES is not None and pid in EXCLUDES:
        print('=> [' + str(pid) + '] Excluded!')
        continue
        
    df = data_df[data_df['id'] == pid]
    solution_state = df['solution_state'].iloc[0]
    size = len(solution_state.split(';'))
        
    if MIN_SIZE is not None and size <= MIN_SIZE:
        print('=> [' + str(pid) + '] Skipped by lower size!')
        continue

    if MAX_SIZE is not None and size >= MAX_SIZE:
        print('=> [' + str(pid) + '] Skipped by upper size!')
        continue

    sdf = sample_df[sample_df['id'] == pid]
    prv_solution = sdf['moves'].iloc[0]
    
    solution = solve(pid, prv_solution)
    if solution is None:
        print('=> [' + str(pid) + '] Failed!')        
    else:
        print('=> [' + str(pid) + '] Success!')        
        
        grows.append({'id': pid, 'moves': solution})

In [ ]:
gdf = None
if len(grows) > 0:
    gdf = pd.DataFrame(grows)
    gdf.to_csv('good.csv', index=False)
gdf

In [ ]:
sub_df = None
if gdf is not None:
    bdf = gdf
    adf = sample_df
    rows = []
    for ri in range(len(adf)):
        pid = adf['id'].iloc[ri]
        moves = adf['moves'].iloc[ri]
        size = len(moves.split('.'))
        df = bdf[bdf['id'] == pid]
        if len(df) > 0:
            moves_b = df['moves'].iloc[0]
            size_b = len(moves_b.split('.'))
            if size_b < size:
                moves = moves_b
                print('=> (' + str(pid) + ') ' + str(size) + ' -> ' + str(size_b))
        rw = {'id': pid, 'moves': moves}
        rows.append(rw)
    sub_df = pd.DataFrame(rows)
    sub_df.to_csv('submission.csv', index=False)
sub_df